<a href="https://colab.research.google.com/github/Lakeserl/Graduation-Thesis/blob/dev_lam/trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install transformers==4.57.1 datasets torch scikit-learn pandas numpy

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json

In [10]:
!pip install transformers datasets torch scikit-learn pandas numpy

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json


training_data = {
    'text': [
        "hey anwar open meeting", "start the meeting", "join call",
        "open video call", "start conference", "begin meeting",
        "khởi động cuộc họp", "mở cuộc gọi", "vào meeting",

        "turn on microphone", "unmute mic", "enable microphone",
        "bật micro", "mở mic", "unmute",
        "hey anwar turn on mic", "activate microphone",

        "turn off microphone", "mute mic", "disable microphone",
        "tắt micro", "tắt mic", "mute",
        "hey anwar mute mic", "silence microphone",

        "turn on camera", "enable camera", "start video",
        "bật camera", "mở camera", "start cam",
        "hey anwar turn on camera", "activate video",

        "turn off camera", "disable camera", "stop video",
        "tắt camera", "tắt cam", "stop video",
        "hey anwar turn off camera", "deactivate video",

        "switch to chat", "open chat", "go to messages",
        "chuyển sang chat", "mở tin nhắn", "vào chat",
        "hey anwar open chat", "show messages",

        "read messages", "read last message", "what's the new message",
        "đọc tin nhắn", "đọc tin nhắn mới", "có tin nhắn gì",
        "hey anwar read messages", "tell me messages",

        "open settings", "go to settings", "show settings",
        "mở cài đặt", "vào settings", "cài đặt",
        "hey anwar open settings", "configuration",

        "emergency", "help me", "sos", "need help",
        "khẩn cấp", "cứu tôi", "help", "emergency call",

        "hello", "how are you", "what time is it",
        "xin chào", "thế nào", "mấy giờ rồi",
        "weather today", "play music"
    ],
    'label': [
        0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2, 2, 2,
        3, 3, 3, 3, 3, 3, 3, 3,
        4, 4, 4, 4, 4, 4, 4, 4,
        5, 5, 5, 5, 5, 5, 5, 5,
        6, 6, 6, 6, 6, 6, 6, 6,
        7, 7, 7, 7, 7, 7, 7, 7,
        8, 8, 8, 8, 8, 8, 8, 8,
        9, 9, 9, 9, 9, 9, 9, 9
    ]
}

intent_labels = {
    0: "OPEN_MEETING",
    1: "MICROPHONE_ON",
    2: "MICROPHONE_OFF",
    3: "CAMERA_ON",
    4: "CAMERA_OFF",
    5: "SWITCH_TO_CHAT",
    6: "READ_MESSAGES",
    7: "OPEN_SETTINGS",
    8: "SOS",
    9: "UNKNOWN"
}

df = pd.DataFrame(training_data)

In [11]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=64)

# Tạo Dataset objects
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IntentDataset(train_encodings, train_labels)
test_dataset = IntentDataset(test_encodings, test_labels)

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(intent_labels)
)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:

test_samples = [
    "hey anwar open meeting",
    "turn on mic",
    "mute camera",
    "read messages",
    "emergency help"
]

print("\n Test :")
for text in test_samples:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    confidence = torch.softmax(outputs.logits, dim=-1)[0][prediction].item()
    print(f"Text: '{text}'")
    print(f"  → Intent: {intent_labels[prediction]} (confidence: {confidence:.2%})")


 Test :
Text: 'hey anwar open meeting'
  → Intent: CAMERA_ON (confidence: 10.92%)
Text: 'turn on mic'
  → Intent: CAMERA_ON (confidence: 11.25%)
Text: 'mute camera'
  → Intent: UNKNOWN (confidence: 11.08%)
Text: 'read messages'
  → Intent: CAMERA_ON (confidence: 11.36%)
Text: 'emergency help'
  → Intent: UNKNOWN (confidence: 11.27%)


In [14]:
import os
os.makedirs("anwar_blind_model", exist_ok=True)

model.save_pretrained("anwar_blind_model")
tokenizer.save_pretrained("anwar_blind_model")

with open("anwar_blind_model/intent_labels.json", "w", encoding="utf-8") as f:
    json.dump(intent_labels, f, ensure_ascii=False, indent=2)

config = {
    "model_name": model_name,
    "num_labels": len(intent_labels),
    "max_length": 64,
    "version": "1.0",
    "trained_date": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
}

with open("anwar_blind_model/config.json", "w", encoding="utf-8") as f:
    json.dump(config, f, ensure_ascii=False, indent=2)


In [15]:
# def predict_intent(text):
#     """Hàm test nhanh intent prediction"""
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
#     outputs = model(**inputs)
#     prediction = torch.argmax(outputs.logits, dim=-1).item()
#     confidence = torch.softmax(outputs.logits, dim=-1)[0][prediction].item()
#     return intent_labels[prediction], confidence

# print("\nTest interactive (nhập 'quit' để thoát):")
# print("Nhập câu lệnh của bạn:")

# test_commands = [
#     "hey anwar bật mic",
#     "tắt camera",
#     "đọc tin nhắn mới",
#     "mở settings",
#     "emergency"
# ]

# for cmd in test_commands:
#     intent, conf = predict_intent(cmd)
#     print(f"'{cmd}' → {intent} ({conf:.2%})")

# print("\n" + "="*50)
# print("TRAINING HOÀN TẤT!")
# print("="*50)
# print("Bước tiếp theo:")